In [13]:
from ultralytics import YOLO
import cv2
import numpy as np
import random
model = YOLO("../models/yolo11n/yolo11nfp16/kaggle/working/weights/yolo11n-cls.onnx")  # load an official model

In [14]:
IMG_PATH = '../images/PotatoEarlyBlight3.jpg'

In [15]:
results=model.predict(IMG_PATH,conf=0.75)

Loading ..\models\yolo11n\yolo11nfp16\kaggle\working\weights\yolo11n-cls.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

image 1/1 e:\Plant Disease\scripts\..\images\PotatoEarlyBlight3.jpg: 640x640 Potato___Early_blight 1.00, Potato___Late_blight 0.00, Potato___healthy 0.00, Soybean___healthy 0.00, Tomato___Early_blight 0.00, 40.5ms
Speed: 43.1ms preprocess, 40.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
# Load image
image = cv2.imread(IMG_PATH)
img_h, img_w = image.shape[:2]

# Get prediction
pred_idx = results[0].probs.top1
pred_label = model.names[pred_idx]
pred_conf = results[0].probs.top1conf
text = f"{pred_label} ({pred_conf*100:.2f}%)"

# Font & styling
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2
text_color = (255, 255, 255)
bg_color = (0, 0, 0)
padding = 10

# Calculate text size
(text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)

# Total label box size
box_w = text_w + 2 * padding
box_h = text_h + baseline + 2 * padding

# Ensure the box fits inside the image
x = 0  # start from left
y = 0  # start from top

# If box is too wide or tall, shrink font (fallback)
while (box_w > img_w or box_h > img_h) and font_scale > 0.3:
    font_scale -= 0.1
    (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    box_w = text_w + 2 * padding
    box_h = text_h + baseline + 2 * padding

# Final positions
x1 = x
y1 = y
x2 = x + box_w
y2 = y + box_h

# Draw background rectangle
cv2.rectangle(image, (x1, y1), (x2, y2), bg_color, -1)

# Put text inside the box
text_x = x1 + padding
text_y = y1 + padding + text_h
cv2.putText(image, text, (text_x, text_y), font, font_scale, text_color, thickness)

# Show result
cv2.imshow("Annotated", image)
cv2.waitKey(0)
cv2.destroyAllWindows()